In [0]:
import numpy as np
import os
from numpy.random import seed
seed(1337)
#from tensorflow import set_random_seed
#set_random_seed(42)
from tensorflow.python.keras.applications import vgg16
from tensorflow.python.keras.applications.vgg16 import preprocess_input
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.python.keras.callbacks import ModelCheckpoint
from tensorflow.python.keras import layers, models, Model, optimizers
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Number of categories
## Images in each category

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
train_data_dir = r"/content/gdrive/My Drive/TrainData"
test_data_dir = r"/content/gdrive/My Drive/ValData"

category_names = sorted(os.listdir(train_data_dir))
nb_categories = len(category_names)
img_pr_cat = []

for category in category_names:
    folder = train_data_dir + '/' + category
    img_pr_cat.append(len(os.listdir(folder)))

In [0]:
img_height, img_width = 64,64
conv_base = vgg16.VGG16(weights='imagenet', include_top=False, pooling='max', input_shape = (img_width, img_height, 3))

58892288/58889256 [==============================] - 1s 0us/step


In [0]:
for layer in conv_base.layers:
    print(layer, layer.trainable)

<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7fa90a2c19e8> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fa8bfe53908> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fa8bfe74160> True
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7fa8bf6338d0> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fa8bf633748> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fa8b05587b8> True
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7fa8b055d0f0> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fa8b055dac8> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fa8b0567dd8> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fa8b056e4a8> True
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7fa8b05763c8> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fa8b057d

In [0]:
  model = models.Sequential()
model.add(conv_base)
model.add(layers.Dense(nb_categories, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense (Dense)                (None, 4)                 2052      
Total params: 14,716,740
Trainable params: 14,716,740
Non-trainable params: 0
_________________________________________________________________


In [0]:
#Number of images to load at each iteration
batch_size = 32# only rescaling
train_datagen =  ImageDataGenerator(rescale=1./255)
test_datagen =  ImageDataGenerator(rescale=1./255)

In [0]:
print('Total number of images for "training":')
train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size = (img_height, img_width),
                                                    batch_size = batch_size, 
                                                    class_mode = "categorical")

print('Total number of images for "testing":')
test_generator = test_datagen.flow_from_directory(test_data_dir,
                                                    target_size = (img_height, img_width),
                                                    batch_size = batch_size,
                                                    class_mode = "categorical",
                                                    shuffle=False)

Total number of images for "training":
Found 2083 images belonging to 4 classes.
Total number of images for "testing":
Found 988 images belonging to 4 classes.


In [0]:
from tensorflow.keras.optimizers import Adam
learning_rate = 5e-5
epochs = 10
checkpoint = ModelCheckpoint("sign_classifier_1.h5", monitor = 'val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=learning_rate, clipnorm = 1.), metrics = ['acc'])


In [0]:
history = model.fit_generator(train_generator, 
                              epochs=epochs, 
                              shuffle=True,
                              validation_data=test_generator,
                              callbacks=[checkpoint]
                              )

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
66/66 [==============================] - ETA: 0s - loss: 0.8350 - acc: 0.6601 
Epoch 00001: val_acc improved from -inf to 0.66599, saving model to sign_classifier_1.h5
66/66 [==============================] - 1759s 27s/step - loss: 0.8350 - acc: 0.6601 - val_loss: 0.7842 - val_acc: 0.6660
Epoch 2/10
66/66 [==============================] - ETA: 0s - loss: 0.5470 - acc: 0.7614
Epoch 00002: val_acc improved from 0.66599 to 0.79150, saving model to sign_classifier_1.h5
66/66 [==============================] - 32s 481ms/step - loss: 0.5470 - acc: 0.7614 - val_loss: 0.5315 - val_acc: 0.7915
Epoch 3/10
66/66 [==============================] - ETA: 0s - loss: 0.4182 - acc: 0.8099
Epoch 00003: val_acc improved from 0.79150 to 0.79453, saving model to sign_classifier_1.h5
66/66 [==============================] - 32s 482ms/step - loss: 0.4182 - acc: 0.8099 - val_loss: 0.6277 - val_acc: 0.7945
Epoch 4/10
66/66 

In [0]:
model = models.load_model("sign_classifier_1.h5")

In [0]:
Y_pred = model.predict_generator(test_generator)
y_pred = np.argmax(Y_pred, axis=1)


Instructions for updating:
Please use Model.predict, which supports generators.


In [0]:
accuracy = accuracy_score(test_generator.classes, y_pred)
print("Accuracy in test set: %0.1f%% " % (accuracy * 100))

Accuracy in test set: 79.5% 


In [0]:
#Download Model
label_class = {0:'BacterialPneumonia',
               1:'COVID-19',
               2:'Normal',
               3:'ViralPneumonia'}
import matplotlib.pyplot as plt
from tensorflow.python.keras.preprocessing import image
def load_image(img_path, show=True):

    img = image.load_img(img_path, target_size=(64, 64))
    img_tensor = image.img_to_array(img)                    # (height, width, channels)
    img_tensor = np.expand_dims(img_tensor, axis=0)         # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
    img_tensor /= 255.                                      # imshow expects values in the range [0, 1]

    if show:
        plt.imshow(img_tensor[0])                           
        plt.axis('off')
        plt.show()

    return img_tensor

test_image = r"/content/gdrive/My Drive/TrainData/COVID-19/_0_1465610.jpeg" #Input Image path
model = models.load_model("sign_classifier_1.h5") #Download and provide the Model path
new_image = load_image(test_image,False)
pred = model.predict(new_image)
#print(pred)
print(label_class[np.argmax(pred[0])]) #return

COVID-19
